# NICE-BC Preprocessing Pipeline

This notebook provides a complete preprocessing pipeline for CT images to prepare them for body composition analysis and treatment response prediction.

## Overview

The pipeline consists of five main steps:
1. **DICOM to NIfTI Conversion**: Converts raw DICOM series to NIfTI format
2. **Image Resampling**: Standardizes spacing, orientation, and origin
3. **HU Windowing**: Truncates intensities and applies mediastinal window
4. **Intensity Normalization**: Normalizes CT intensity values for consistent processing
5. **Body Composition Segmentation**: Uses nnU-Net models to segment body composition components

## Prerequisites

- Python packages: `SimpleITK`, `monai`, `tqdm`, `numpy`
- Access to AutoPanoM v1.0.0.0 models (contact Prof. Zhenwei Shi for access)
- nnU-Net prediction modules in `nnunet_projects_BC/` and `nnunet_projects_Bone/` directories

## Directory Structure

Before running, ensure your data is organized as follows:
```
preprocessing/
├── 0_DICOM/          # Input: Patient subdirectories with DICOM files
│   ├── patient_001/
│   ├── patient_002/
│   └── ...
├── 1_NII/            # Generated: NIfTI format images
├── 2_Res/            # Generated: Resampled images
├── 3_HU/             # Generated: HU windowed images
├── 4_Norm/           # Generated: Normalized images + segmentation masks
├── 5_BC/             # Generated: Body composition masks
└── 6_Bone/           # Generated: Bone segmentation masks
```

## Usage

Execute the cells sequentially. Each step processes all patients in the input directory and saves results to the corresponding output directory.


In [ ]:
from pathlib import Path
import SimpleITK as sitk
from tqdm import tqdm
import shutil
import numpy as np

In [ ]:
# Define directory paths for the preprocessing pipeline
# Each directory represents a different stage of the preprocessing workflow
dcm_dir = Path("./0_DICOM")      # Input: Raw DICOM files organized by patient
og_dir = Path("./1_NII")         # Output: Converted NIfTI format images
res_dir = Path("./2_Res")        # Output: Resampled images (1mm spacing, LPS orientation)
hu_dir = Path("./3_HU")          # Output: HU windowed images (truncated and windowed)
norm_dir = Path("./4_Norm")      # Output: Normalized images ready for segmentation
bc_dir = Path("./5_BC")          # Output: Body composition segmentation masks
bone_dir = Path("./6_Bone")      # Output: Bone segmentation masks

## Step 1: DICOM to NIfTI Conversion

This step converts DICOM series to NIfTI format for easier processing. The function automatically:
- Detects all DICOM series in each patient directory
- Selects the series with the most files (in case multiple series exist)
- Converts the DICOM series to a single 3D NIfTI image

**Input**: `0_DICOM/` directory with patient subdirectories containing DICOM files  
**Output**: `1_NII/` directory with patient subdirectories containing `img.nii.gz` files

In [ ]:
def read_dicom_series(series_directory: Path) -> sitk.Image:
    reader = sitk.ImageSeriesReader()
    series_ids = reader.GetGDCMSeriesIDs(str(series_directory))
    if not series_ids:
        raise FileNotFoundError(f"No DICOM series found in {series_directory}")

    # Pick the series with the most files if multiple exist in the folder
    best_file_names = []
    for series_id in series_ids:
        file_names = reader.GetGDCMSeriesFileNames(str(series_directory), series_id)
        if len(file_names) > len(best_file_names):
            best_file_names = file_names

    reader.SetFileNames(best_file_names)
    image = reader.Execute()
    return image


for patient_dir in dcm_dir.iterdir():
    if not patient_dir.is_dir():
        continue

    img_sitk = read_dicom_series(patient_dir)
    dst_dir = og_dir / patient_dir.name
    dst_dir.mkdir(parents=True, exist_ok=True)
    sitk.WriteImage(img_sitk, str(dst_dir / "img.nii.gz"))

    print(f"{patient_dir.name} size: {img_sitk.GetSize()}, spacing: {img_sitk.GetSpacing()}")

## Step 2: Image Resampling and Spatial Standardization

This step standardizes the spatial properties of all images to ensure consistency:
- **Spacing**: Resamples to 1mm × 1mm × 1mm isotropic voxel spacing using B-spline interpolation
- **Direction**: Reorients images to LPS (Left-Posterior-Superior) coordinate system
- **Origin**: Sets image origin to (0, 0, 0) for consistent spatial reference

**Input**: `1_NII/` directory with `img.nii.gz` files  
**Output**: `2_Res/` directory with resampled and standardized `img.nii.gz` files

**Note**: The resampling uses B-spline interpolation for intensity images. For mask images, nearest-neighbor interpolation is used to preserve discrete label values.

In [ ]:
def resample(in_path, out_path, out_spacing=[1.0, 1.0, 1.0], is_mask = False):
    image_3d = sitk.ReadImage(str(in_path))

    """ 1. spacing """
    original_size = image_3d.GetSize()
    original_spacing = image_3d.GetSpacing()
    same_spacing = True
    for i in range(len(original_spacing)):
        if original_spacing[i] != out_spacing[i]:
            same_spacing = False
            break
    if not same_spacing:
        out_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, out_spacing)]
        resample_filter = sitk.ResampleImageFilter()
        resample_filter.SetOutputSpacing(out_spacing)
        resample_filter.SetSize(out_size)
        resample_filter.SetOutputDirection(image_3d.GetDirection())
        resample_filter.SetOutputOrigin(image_3d.GetOrigin())
        resample_filter.SetTransform(sitk.Transform())
        # resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

        if is_mask: # 如果是mask图像，就选择sitkNearestNeighbor这种插值
            resample_filter.SetInterpolator(sitk.sitkNearestNeighbor)
        else: # 如果是普通图像，就采用sitkBSpline插值法
            resample_filter.SetInterpolator(sitk.sitkBSpline)

        image_3d = resample_filter.Execute(image_3d)

    """ 2. direction """
    original_direction = image_3d.GetDirection()
    out_direction = (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
    same_direction = True
    for i in range(len(original_direction)):
        if original_direction[i] != out_direction[i]:
            same_direction = False
            break
    image_3d = sitk.DICOMOrient(image_3d, "LPS")
    image_3d.SetDirection(out_direction)

    """ 3. origin """
    original_origin = image_3d.GetOrigin()
    out_origin = (0.0, 0.0, 0.0)
    same_origin = True
    for i in range(len(original_origin)):
        if original_origin[i] != out_origin[i]:
            same_origin = False
    image_3d.SetOrigin(out_origin)

    """ 4. write """
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    sitk.WriteImage(image_3d, str(out_path))


for patient_dir in tqdm(list(og_dir.iterdir())):
    if not patient_dir.is_dir():
        continue

    img_path = patient_dir / "img.nii.gz"
    if not img_path.exists():
        raise Exception(f"{img_path} not exists")

    dst_img_path = res_dir / patient_dir.name / "img.nii.gz"
    dst_img_path.parent.mkdir(parents=True, exist_ok=True)

    if not dst_img_path.exists():
        resample(img_path, dst_img_path, out_spacing=[1.0, 1.0, 1.0], is_mask=False)

## Step 3: HU Windowing

This step applies HU (Hounsfield Unit) windowing to the CT images based on the anatomical focus of this study:
- **Truncation**: Clips intensities to the range of [-1024, 1024] HU
- **Mediastinal Window**: Applies a standard mediastinal imaging window with:
  - Window Level (WL) = 50 HU
  - Window Width (WW) = 350 HU
  - Window range: [WL - WW/2, WL + WW/2] = [-125, 225] HU

The windowing process clips values outside the window range and prepares the images for normalization.

**Input**: `2_Res/` directory with resampled `img.nii.gz` files  
**Output**: `3_HU/` directory with HU windowed `img.nii.gz` files

**Note**: This step ensures that the intensity values are within the appropriate range for mediastinal imaging and removes extreme outliers before normalization.

In [ ]:
def apply_hu_windowing(in_path, out_path, 
                       truncate_range=[-1024, 1024],
                       window_level=50, window_width=350):
    """
    Apply HU windowing to CT images:
    1. Truncate intensities to the specified range
    2. Apply mediastinal window (window level and width)
    
    Parameters:
    -----------
    in_path : Path
        Input image path
    out_path : Path
        Output image path
    truncate_range : list
        [min_hu, max_hu] for truncation
    window_level : float
        Window level in HU (default: 50 HU for mediastinal window)
    window_width : float
        Window width in HU (default: 350 HU for mediastinal window)
    """
    img = sitk.ReadImage(str(in_path))
    img_arr = sitk.GetArrayFromImage(img)
    
    # 1. Truncate to [-1024, 1024] HU
    img_arr = np.clip(img_arr, truncate_range[0], truncate_range[1])
    
    # 2. Apply mediastinal window
    # Window range: [WL - WW/2, WL + WW/2]
    window_min = window_level - window_width / 2.0
    window_max = window_level + window_width / 2.0
    img_arr = np.clip(img_arr, window_min, window_max)
    
    # Convert back to SimpleITK image
    img_windowed = sitk.GetImageFromArray(img_arr)
    img_windowed.CopyInformation(img)
    
    # Write output
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    sitk.WriteImage(img_windowed, str(out_path))


for patient_dir in tqdm(list(res_dir.iterdir())):
    if not patient_dir.is_dir():
        continue

    img_path = patient_dir / "img.nii.gz"
    if not img_path.exists():
        raise Exception(f"{img_path} not exists")

    dst_img_path = hu_dir / patient_dir.name / "img.nii.gz"
    dst_img_path.parent.mkdir(parents=True, exist_ok=True)

    if not dst_img_path.exists():
        apply_hu_windowing(
            img_path, 
            dst_img_path,
            truncate_range=[-1024, 1024],
            window_level=50,
            window_width=350
        )

## Step 4: Intensity Normalization

This step normalizes the intensity values of the CT images using MONAI's `NormalizeIntensity` transform, which performs z-score normalization (mean=0, std=1) to standardize the intensity distribution across different scanners and acquisition parameters.

**Input**: `3_HU/` directory with HU windowed `img.nii.gz` files  
**Output**: `4_Norm/` directory with normalized `img.nii.gz` files

In [ ]:
import monai

trans = monai.transforms.NormalizeIntensity()

for patient_dir in tqdm(list(hu_dir.iterdir())):
    if not patient_dir.is_dir():
        continue

    img_path = patient_dir / "img.nii.gz"
    if not img_path.exists():
        raise Exception(f"{img_path} not exists")

    dst_img_path = norm_dir / patient_dir.name / "img.nii.gz"
    dst_img_path.parent.mkdir(parents=True, exist_ok=True)

    if not dst_img_path.exists():
        img = sitk.ReadImage(str(img_path))
        img_arr = sitk.GetArrayFromImage(img)[None]
        img_norm = trans(img_arr)
        img_norm = img_norm[0]
        img_norm = sitk.GetImageFromArray(img_norm)
        img_norm.CopyInformation(img)
        sitk.WriteImage(img_norm, str(dst_img_path))

## Step 5: Body Composition Segmentation using nnU-Net

This step performs automated segmentation using pre-trained nnU-Net models. The pipeline includes two separate segmentation tasks:

### 5.1 Body Composition (BC) Segmentation

**Input**: `4_Norm/` directory with normalized `img.nii.gz` files  
**Output**: `5_BC/` directory with body composition masks

### 5.2 Bone Segmentation
Segments bone structures (vertebrae) for anatomical reference.

**Input**: `4_Norm/` directory with normalized `img.nii.gz` files  
**Output**: `6_Bone/` directory with bone segmentation masks

**Note**: 
- The segmentation models require access to the AutoPanoM v1.0.0.0 models (see main README for access information)
- The `nnunet_projects_BC/` and `nnunet_projects_Bone/` directories must contain the prediction modules

In [ ]:
import sys
sys.path.append("./nnunet_projects_BC/")
import predict_BC

for patient_dir in norm_dir.iterdir():
    if not patient_dir.is_dir():
        continue

    img_path = patient_dir / "img.nii.gz"
    if not img_path.exists():
        raise Exception(f"{img_path} not exists")

    mask_path = bc_dir / "mask.nii.gz"
    if mask_path.exists():
        continue
    
    predict_BC.main_worker(img_path, mask_path)
    bc_dir.mkdir(parents=True, exist_ok=True)
    shutil.copy2(mask_path, bc_dir / patient_dir.name / mask_path.name)

In [ ]:
import sys
sys.path.append("./nnunet_projects_Bone/")
import predict_Bone

for patient_dir in norm_dir.iterdir():
    if not patient_dir.is_dir():
        continue

    img_path = patient_dir / "img.nii.gz"
    if not img_path.exists():
        raise Exception(f"{img_path} not exists")

    mask_path = bone_dir / "mask.nii.gz"
    if mask_path.exists():
        continue
    
    predict_Bone.main_worker(img_path, mask_path)
    bone_dir.mkdir(parents=True, exist_ok=True)
    shutil.copy2(mask_path, bone_dir / patient_dir.name / mask_path.name)